# Assignment 2
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

## Exercise 2.1

In [2]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator

In [3]:
# Load data
stores = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

### Classes, helper functions and global variables

#### Locations

In [4]:
class Location:
    
    # Set coordinates of headquarters as class variable
    coords_hq = stores.loc[0, 'Lat'], stores.loc[0, 'Long']
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        
    @property
    def coords(self):
        """Defines coordinates of a location"""
        return (self.lat, self.long)
    
    @property
    def distance_hq(self):
        """Defines distance from location to headquarters"""
        dist = round(haversine(self.coords_hq, self.coords))
        return dist
    
    @property
    def is_jumbo(self):
        """Boolean value that shows if location is a Jumbo location"""
        return True if self.store_type=='Jumbo' else False

In [5]:
def get_params(df, index):
    """Function to get all necessary variables from stores dataframe to instantiate a Location instance for every store"""
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

In [6]:
class Route:
   
    # Set class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store, hq):
        self.inner_route=[]
        self.farthest_store = farthest_store
        self.hq = hq
    
    @property
    def meeting_time(self):
        """Defines the meeting time at individiual stores in the route"""
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def store_types(self):
        """Defines the type of stores in this route"""
        return 'Jumbo' if self.farthest_store.is_jumbo else 'Coop or other'
    
    @property
    def full_route(self):
        """Defines full route of the route, including the start and finish at headquarters"""
        full_route = self.inner_route.copy()
        full_route.insert(0, self.hq)
        full_route.append(self.hq)
        return full_route
    
    @property
    def total_distance(self):
        """Defines total distance in km of the route"""
        return sum(Route.distances(self.full_route))
    
    @property
    def cumsum(self):
        """Defines cumulative sum of distances at each location"""
        cumsum = list(np.cumsum(Route.distances(self.full_route)))
        cumsum.insert(0, 0)
        return cumsum
    
    @property
    def total_visit_time(self):
        """Defines total time spend in all stores of route together"""
        return len(self.inner_route) * self.meeting_time
    
    @property
    def working_hours_constraint(self):
        """Defines if route meets the constraint of John's working hours"""
        return (Route.travel_time(self.full_route) + self.total_visit_time) <= Route.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
        return (Route.travel_time(self.inner_route) + self.total_visit_time) <= Route.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        """Defines if route meets both constraints John's working hours and visiting hours at every store"""
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @staticmethod
    def calc_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        dist = round(haversine(loc1.coords, loc2.coords))
        return dist
    
    @staticmethod
    def distances(route):
        """Defines all distances between consecutive locations in a route"""
        distances = [Route.calc_dist(route[idx], route[idx+1]) for 
                          idx, _ in enumerate(route[:-1])]
        return distances
    
    @staticmethod
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (Route.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    @staticmethod
    def travel_time(route):
        """
        Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
        """
        minutes_between_locs = [Route.dist_to_min(dist) for dist in Route.distances(route)]
        travel_time = sum(minutes_between_locs)
        return travel_time
    
    def insert(self, pos, item):
        """Insert location at given position in route"""
        self.inner_route.insert(pos, item)
    
    def remove(self, item):
        """Remove location from route"""
        self.inner_route.remove(item)

In [13]:
class RoutePlanner:
    
    def __init__(self, locations):
        self.locations = locations[1:]
        self.hq = locations[0]
        
    @property
    def loc_distances_hq(self):
        """Defines the list of stores sorted in decreasing order based on distance from headquarter"""
        sorted_dist = sorted(self.locations, key=lambda x: x.distance_hq, reverse=True)
        return sorted_dist
    
    @property
    def to_visit(self):
        """Defines the list of stores that still need to be visited in a future route"""
        stores_to_visit = [loc for loc in self.locations if loc.visited==False]
        return stores_to_visit
    
    @property
    def current_farthest_store(self):
        farthest_store = [loc for loc in self.loc_distances_hq if loc.visited==False][0]
        return farthest_store
    
    @property
    def stores_left(self):
        """Defines if there are any stores left to visit"""
        return len([loc for loc in self.locations if loc.visited==False and loc!= self.hq])>0
    
    @staticmethod
    def calc_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        dist = round(haversine(loc1.coords, loc2.coords))
        return dist
    
    def potential_stores(self, farthest_store):
        """Defines potential stores for a given route with given farthest location"""
        if farthest_store.is_jumbo:
            potential_stores = [loc for loc in self.to_visit if loc.store_type=='Jumbo' and loc!=farthest_store]
        else: 
            potential_stores = [loc for loc in self.to_visit if loc.store_type!= 'Jumbo' and loc!=farthest_store]
        return potential_stores

    def dist_farthest_store(self, farthest_store):
        """Defines distances for all potential stores from farthest store in a route in sorted order ascending"""
        distances = [(RoutePlanner.calc_dist(farthest_store, potential_store), potential_store) 
                     for potential_store in self.potential_stores(farthest_store)]
        distances.sort(key=operator.itemgetter(0))
        sorted_dist = [potential_store for dist, potential_store in distances]
        return sorted_dist
    
    def plan_route(self, route):
        """Plans individual routes"""
        
        # Insert current farthest store to be visited in rote
        route.insert(0, route.farthest_store)
        
        # All potential locations in ascending order
        for i in self.dist_farthest_store(route.farthest_store):
            best_dist = math.inf
            best_pos = None
            
            # Try location on every position in route
            for pos in range(len(route.inner_route)+1):
                
                # Insert location at position
                route.insert(pos, i)
                
                # Check if location at given position meets constraints and;
                # Check if this position is better than all previous viable positions for this location
                if route.all_hours_constraints and route.total_distance<best_dist:
                    best_dist = route.total_distance
                    best_pos = pos
                
                # Remove location from route
                route.remove(i)
            
            # If there is a viable position, insert location at best viable position in route
            if best_pos!=None:
                route.insert(best_pos, i)
        
        # Set all locations that are inserted to route to visited
        for loc in route.inner_route:
            loc.visited = True
            
    def solve(self):
        """Solves VRP for all locations"""
        
        # Set route number
        j = 0
        routes = {} # Empty dict to store routes
        while self.stores_left:
            j += 1 # Update route number
            farthest_store = self.current_farthest_store # Assign current farthest store
            route = Route(farthest_store, self.hq) # Instantiate route instance with current farthest store
            self.plan_route(route) # Plan this route
            routes[j] = route # Store route with route as key in dict
        return routes

In [8]:
def to_Excel(planned_routes, save=True, file_name=None):
    """Creates dataframe with needed information from solved VRP, saves to Excel file if needed"""
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr_route in planned_routes.items():
        nr = nr_route[0]
        route = nr_route[1]
        cumsum_loc = 0  # Define current cumulative sum location
        
        # Add data for location to dictionary
        for location in route.full_route:
            data['Route Nr.'].append(nr)
            data['City Nr.'].append(location.nr)
            data['City Name'].append(location.name)
            data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
            data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
            
            cumsum_loc += 1 # Update current cumulative sum location

        total_km_at_start += route.cumsum[-1] # Update total kilometers traveled at start of the route in previous routes
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

In [8]:
# Instantiate location instance for every store
locations = [Location(*get_params(stores, index)) for index, _ in stores.iterrows()]

# Instantiate planner object
planner = RoutePlanner(locations)
planned_routes = planner.solve() # Solve VRP

df_q21 = to_Excel(planned_routes, file_name='../excel-files/Ex2.1-1273195.xls')

In [9]:
df_q21

,Route Nr.,City Nr.,City Name,Total Distance in Route (km),Total Distance (km)
0,1,0,EMTE HEADQUARTERS VEGHEL,0,0
1,1,7,EMTE BORCULO,89,89
2,1,40,EMTE HAAKSBERGEN JHR V HEIJDEN,104,104
3,1,41,EMTE HAAKSBERGEN F BOLSTRAAT,105,105
4,1,96,EMTE RIJSSEN OPSTALLSTR,126,126
5,1,34,EMTE ENTER FR,130,130
6,1,75,EMTE NIJVERDAL PORTLANDWEG,139,139
7,1,127,EMTE VRIEZENVEEN,151,151
8,1,48,EMTE HENGELO P STEENB,171,171
9,1,65,EMTE LOSSER,183,183


In [10]:
[route.travel_time(route.full_route) + route.total_visit_time for route in planned_routes.values()]

[569, 593, 520, 578, 497, 514, 480, 504, 458, 457, 297]

In [12]:
[route.total_distance for route in planned_routes.values()]

[463, 347, 419, 285, 296, 232, 238, 169, 294, 191, 85]